In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')


In [ ]:
files.upload()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
kidney_df = pd.read_csv('/content/drive/MyDrive/dataset/kidney_disease.csv')

In [ ]:
kidney_df.head()

In [ ]:
kidney_df.info()

In [ ]:
kidney_df.isnull().sum()

In [ ]:
#imputing null values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df_imputed = pd.DataFrame(imputer.fit_transform(kidney_df), columns=kidney_df.columns)
df_imputed.head()

In [ ]:
df_imputed.isnull().sum()

In [ ]:
#finding unique values in columns
for col in df_imputed.columns:
  print(f'{col}: {df_imputed[col].unique()}')

In [ ]:
print(df_imputed["rc"].mode())
print(df_imputed["wc"].mode())
print(df_imputed["pcv"].mode())

In [ ]:
df_imputed["classification"]=df_imputed["classification"].replace("ckd\t","ckd")
df_imputed["classification"]=df_imputed["classification"].replace("notckd","not ckd")

df_imputed["cad"]=df_imputed["cad"].replace("\tno","no")

df_imputed["dm"]=df_imputed["dm"].replace( "\tno","no")
df_imputed["dm"]=df_imputed["dm"].replace( "\tyes","yes")
df_imputed["dm"]=df_imputed["dm"].replace( " yes","yes")

df_imputed["wc"]=df_imputed["wc"].replace("\t6200","9800")
df_imputed["wc"]=df_imputed["wc"].replace("\t?","5.2")
df_imputed["wc"]=df_imputed["wc"].replace("\t8400","9800")

df_imputed["pcv"]=df_imputed["pcv"].replace("\t43","41")
df_imputed["pcv"]=df_imputed["pcv"].replace("\t?","41")

df_imputed["rc"]=df_imputed["rc"].replace("\t?","5.2")



In [ ]:
#finding unique values in columns
for col in df_imputed.columns:
  print(f'{col}: {df_imputed[col].unique()}')

In [ ]:
df_imputed.dtypes

In [ ]:
kidney_df.select_dtypes(exclude=["object"]).columns

In [ ]:
for i in kidney_df.select_dtypes(exclude=["object"]).columns:
  df_imputed[i] = df_imputed[i].astype('float')

In [ ]:
df_imputed.dtypes

In [ ]:
object_dtypes = df_imputed.select_dtypes(include = 'object')
object_dtypes.head()

In [ ]:
dictonary = {
        "rbc": {
        "abnormal":1,
        "normal": 0,
    },
        "pc":{
        "abnormal":1,
        "normal": 0,
    },
        "pcc":{
        "present":1,
        "notpresent":0,
    },
        "ba":{
        "notpresent":0,
        "present": 1,
    },
        "htn":{
        "yes":1,
        "no": 0,
    },
        "dm":{
        "yes":1,
        "no":0,
    },
        "cad":{
        "yes":1,
        "no": 0,
    },
        "appet":{
        "good":1,
        "poor": 0,
    },
        "pe":{
        "yes":1,
        "no":0,
    },
        "ane":{
        "yes":1,
        "no":0,
    }
}

In [ ]:
df=df_imputed.replace(dictonary)
df.head()

In [ ]:
X = df.drop(['id','classification', 'sg', 'appet', 'rc', 'pcv', 'hemo', 'sod'], axis = 1)
y = df['classification']
X.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)
model.fit(X_train, y_train)

In [ ]:
y_pred= model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(f"Accuracy is {round(accuracy_score(y_test, model.predict(X_test))*100, 2)}%")

In [ ]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print('Accuracy on Training data : ', training_data_accuracy)

In [ ]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)
print('Accuracy on Test data : ', test_data_accuracy)

In [ ]:
df=pd.DataFrame({'Actual':y_test,'Predicted':X_test_prediction})
df.head()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X.tail()

In [ ]:
input_data = (58,80,0,0,0,0,0,0,131,18,1.1,3.5,6800,0,0,0,0,0)

# change the input data to a numpy array
input_data_as_numpy_array= np.asarray(input_data)

# reshape the numpy array as we are predicting for only on instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = model.predict(std_data)
print(prediction)

if (prediction[0]== "not ckd"):
  print('The Person does not have a chronic kidney Disease')
else:
  print('The Person has chronic kindey Disease')